In [9]:
import numpy as np
import pandas as pd
import pandas_datareader as web    #to get the stock prices
from datetime import datetime
import seaborn as sn
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

from numba import cuda 
device = cuda.get_current_device()

from keras.models import Sequential  
from keras.layers import Dense        
from keras.layers import LSTM        
from keras.layers import Dropout     

In [10]:
start = datetime(1978, 1, 1)
end = datetime.now()

df = web.DataReader("AAPL", 'yahoo', start, end)
df = df.reset_index()
df = df[['Date', 'Close','Open','Low','High']]
df.describe()

,Close,Open,Low,High
count,10147.000000,10147.000000,10147.000000,10147.000000
mean,10.500621,10.499114,10.381170,10.612525
std,21.174792,21.176955,20.910248,21.422848
min,0.049107,0.049665,0.049107,0.049665
25%,0.272768,0.274052,0.266395,0.281250
50%,0.446429,0.446429,0.437500,0.453125
75%,11.911250,11.924285,11.810536,12.005000
max,143.160004,143.600006,141.369995,145.089996


## Seperate Data

In [11]:
properties = list(df.columns.values)
properties.remove('Close')
properties.remove('Date')
print(properties)
X = df[properties]
y = df['Close']

['Open', 'Low', 'High']


In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1)

In [13]:
model = Sequential()

model.add( LSTM (units = 50, return_sequences = True , input_shape = ( X_train.shape[1],1 ) ) )
model.add(Dropout(0.2))

model.add( LSTM (units = 50, return_sequences = True ) )
model.add(Dropout(0.2))

model.add( LSTM (units = 50, return_sequences = True ) )
model.add(Dropout(0.2))

model.add( LSTM (units = 50) )
model.add(Dropout(0.2))

model.add(Dense(units = 1))

model.compile(optimizer = 'adam', loss = 'mean_squared_error')

history = model.fit(X_train, y_train, epochs = 100, batch_size = 128,validation_data=(X_val, y_val))

device.reset() ## Clear GPU Memory

Epoch 1/100


ValueError: in user code:

    /home/matthew/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:571 train_function  *
        outputs = self.distribute_strategy.run(
    /home/matthew/anaconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/matthew/anaconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/matthew/anaconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    /home/matthew/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:531 train_step  **
        y_pred = self(x, training=True)
    /home/matthew/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/base_